In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core import StorageContext, VectorStoreIndex, Settings, Document
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.node_parser import SentenceSplitter



class ChatEngine():
    def __init__(self, llm , docs: list[Document] = [],  collection_name: str = "docs", embed_model = HuggingFaceEmbedding(), token_limit :int = 8000, chroma_db_path: str = "./chroma_db"):
        
        self.docs = docs
        self.collection_name = collection_name
        self.chroma_db_path = chroma_db_path
        self.embed_model = embed_model
        self.token_limit = token_limit
        self.splitter = SentenceSplitter(chunk_size=512, chunk_overlap=200)
        # Lưu tạm vào Chroma 
        chroma_client = chromadb.PersistentClient(path= chroma_db_path)
        
       
        nodes = self.splitter.get_nodes_from_documents(docs_a)

        # 3. Nếu collection đã tồn tại, load lại. Nếu chưa, tạo mới.
        chroma_collection = chroma_client.get_or_create_collection(collection_name)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        self.index = VectorStoreIndex(
            nodes, storage_context=storage_context, embed_model=embed_model, store_nodes_override=True
        )
        Settings.llm = llm                   # vô hiệu hóa LLM dùng OpenAI
        self.memory = ChatMemoryBuffer.from_defaults(token_limit=token_limit)
        self.chat_engine = self.index.as_chat_engine(chat_mode="context", memory=self.memory)
    def chat(self, message):
        if self.chat_engine:
            response = self.chat_engine.chat(message)
            return response
        else:
            return "Lỗi khi tạo mô hình"
    def chat2(self, message):
        if self.chat_engine:
            query_engine = self.index.as_query_engine(similarity_top_k=5)
            result = query_engine.query(message)
            print("🔎 Các đoạn được truy xuất:")
            for node in result.source_nodes:
                print("-" * 80)
                print(node.node.text[:500])
            return result.response
        else:
            return "Lỗi khi tạo mô hình"

    def update_docs(self, new_docs: list[Document]):
        self.docs.extend(new_docs)
        nodes = self.splitter.get_nodes_from_documents(new_docs)
        self.index.insert_nodes(nodes)
        self.chat_engine = self.index.as_chat_engine(chat_mode='context', memory=self.memory)

        

from llama_index.core import VectorStoreIndex, Document
from llama_index.readers.file import PyMuPDFReader,DocxReader, PandasExcelReader 


def read_uploaded_file(file_path) -> list[Document] | None:

    filename = file_path.name if hasattr(file_path, "name") else str(file_path)
    ext = filename.lower().split('.')[-1]
    with open(file_path, "rb") as f:
        file_bytes = f.read()

    if ext == "pdf":
        pdf_loader = PyMuPDFReader()
        documents = pdf_loader.load(file_path=file_path)
    elif ext == "docx":
        docx_loader = DocxReader()
        documents = docx_loader.load_data(file  = file_path)
    elif ext in ["xls", "xlsx"]:
        excel_loader = PandasExcelReader()
        documents = excel_loader.load_data(file = file_path)
    elif ext == "txt":
        text = file_bytes.decode("utf-8")
        documents =  [Document(text=text, metadata={"filename": filename})]
    else:
        return None

    return documents


import os
from dotenv import load_dotenv
load_dotenv()
from llama_index.llms.openai_like import OpenAILike
from llama_index.llms.openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

# llm=OpenAILike(model="qwen/qwen3-coder:free",
#         api_key = api_key,
#         temperature = 0.0,

#         api_base = "https://openrouter.ai/api/v1",
#         is_chat_model=True,
#         is_function_calling_model=True, 
# )
llm = OpenAI(model="gpt-4.1-2025-04-14",
             temperature=0
             )

embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2", max_length=8000)
docs_a = read_uploaded_file(r"C:\Users\ACER\Desktop\nhan xet.docx")

new_doc = read_uploaded_file(r"C:\Users\ACER\Desktop\BTVN\BTVN GB12.xlsx")
chatEngine = ChatEngine(llm=llm,embed_model=embed_model, collection_name="final_9")
chatEngine.update_docs(new_doc)
chatEngine.update_docs(docs_a)





f:\nexlab\AI project 2\SmarDocChatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0731 10:53:43.675000 20364 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core import StorageContext, VectorStoreIndex, Settings, Document
from llama_index.core.memory import ChatMemoryBuffer


class ChatEngine():
    def __init__(self, llm , collection_name: str = "docs", embed_model = HuggingFaceEmbedding("all-MiniLM-L6-v2"), token_limit :int = 4000, chroma_db_path: str = "./chroma_db"):
        
        self.docs = []
        self.collection_name = collection_name
        self.chroma_db_path = chroma_db_path
        self.embed_model = embed_model
        self.token_limit = token_limit
        self.llm = llm
        self.memory = ChatMemoryBuffer.from_defaults(token_limit=self.token_limit)
        Settings.llm = self.llm                   # vô hiệu hóa LLM dùng OpenAI
        self.chat_engine = None
    def load_old_docs(self):
        chroma_client = chromadb.PersistentClient(path=self.chroma_db_path)
        chroma_collection = chroma_client.get_collection(self.collection_name)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        self.index = VectorStoreIndex.from_vector_store(
            vector_store=vector_store, storage_context=storage_context, embed_model=self.embed_model, store_nodes_override=True
        )
        self.chat_engine = self.index.as_chat_engine(chat_mode="condense_plus_context", memory=self.memory)

    def create_new_index(self):
        chroma_client = chromadb.PersistentClient(path=self.chroma_db_path)
        chroma_collection = chroma_client.create_collection(self.collection_name)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        self.index = VectorStoreIndex.from_documents(
            self.docs, storage_context=storage_context, embed_model=self.embed_model)
        Settings.llm = self.llm                   # vô hiệu hóa LLM dùng OpenAI
        self.memory = ChatMemoryBuffer.from_defaults(token_limit=self.token_limit)
        self.chat_engine = self.index.as_chat_engine(chat_mode="condense_plus_context", memory=self.memory)
            
        
    def load_docs(self,new_docs: list[Document]):
        self.docs = new_docs
        chroma_client = chromadb.PersistentClient(path=self.chroma_db_path)
        existing_collections = [c.name for c in chroma_client.list_collections()]
        if self.collection_name in existing_collections:
            self.load_old_docs()
            self.index.refresh_ref_docs(new_docs)
            self.chat_engine = self.index.as_chat_engine(chat_mode='condense_plus_context', memory=self.memory)
        else:
            self.create_new_index()
            
    def chat(self, message):
        if self.chat_engine:
            response = self.chat_engine.chat(message)
            return response
        else:
            return "Lỗi khi tạo mô hình"

    def update_docs(self, new_docs: list[Document]):
        self.docs.extend(new_docs)
        self.index = self.index.from_documents(self.docs)
        self.chat_engine = self.index.as_chat_engine(chat_mode='condense_plus_context', memory=self.memory)
        


f:\nexlab\AI project 2\SmarDocChatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0731 23:44:35.634000 12944 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.readers.file import PyMuPDFReader,DocxReader, PandasExcelReader 


def read_uploaded_file(file_path) -> list[Document] | None:

    filename = file_path.name if hasattr(file_path, "name") else str(file_path)
    ext = filename.lower().split('.')[-1]
    with open(file_path, "rb") as f:
        file_bytes = f.read()

    if ext == "pdf":
        pdf_loader = PyMuPDFReader()
        documents = pdf_loader.load(file_path=file_path)
    elif ext == "docx":
        docx_loader = DocxReader()
        documents = docx_loader.load_data(file  = file_path)
    elif ext in ["xls", "xlsx"]:
        excel_loader = PandasExcelReader()
        documents = excel_loader.load_data(file = file_path)
    elif ext == "txt":
        text = file_bytes.decode("utf-8")
        documents =  [Document(text=text, metadata={"filename": filename})]
    else:
        return None

    return documents



In [3]:


import os
from dotenv import load_dotenv
load_dotenv()
from llama_index.llms.openai_like import OpenAILike
from llama_index.llms.openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

# llm=OpenAILike(model="qwen/qwen3-coder:free",
#         api_key = api_key,
#         temperature = 0.0,

#         api_base = "https://openrouter.ai/api/v1",
#         is_chat_model=True,
#         is_function_calling_model=True, 
# )
llm = OpenAI(model="gpt-4.1-2025-04-14",
             temperature=0
             )
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
docs_a = read_uploaded_file(r"C:\Users\ACER\Desktop\nhan xet.docx")

new_doc = read_uploaded_file(r"C:\Users\ACER\Desktop\BTVN\BTVN GB12.xlsx")
chatEngine = ChatEngine(llm=llm,embed_model=embed_model, collection_name="final2", chroma_db_path="./db", token_limit=1024)



In [4]:
chroma_client = chromadb.PersistentClient(path=chatEngine.chroma_db_path)
existing_collections = [c.name for c in chroma_client.list_collections()]
if chatEngine.collection_name in existing_collections:
    chroma_client = chromadb.PersistentClient(path=chatEngine.chroma_db_path)
    chroma_collection = chroma_client.get_collection(chatEngine.collection_name)
    vector_store = ChromaVectorStore.from_collection(chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store, storage_context=storage_context, embed_model=chatEngine.embed_model
    )
    chat = chatEngine.index.as_chat_engine(chat_mode="context", memory=chatEngine.memory)

AttributeError: 'ChatEngine' object has no attribute 'index'

In [6]:
res = chatEngine.chat("bạn cho mình xin thông tin điểm của bạn Gia Bảo với")
res

AgentChatResponse(response='Dưới đây là thông tin điểm của bạn Hà Gia Bảo:\n\n**Bảng điểm tổng hợp:**\n- Họ và tên: Hà Gia Bảo\n- Rank: 2\n- Điểm cộng: 1.0\n- Lý thuyết (TN): 3.5\n- Thực hành (TH): 5.0\n- Ghi chú (note): 1 5 7\n- Tổng điểm: 9.5\n\n**Chi tiết điểm các buổi (nếu cần):**\nHiện tại trong bảng chi tiết các buổi, không thấy có dòng điểm tương ứng với Hà Gia Bảo. Nếu bạn cần chi tiết từng buổi, vui lòng xác nhận lại hoặc cung cấp thêm thông tin.\n\nNếu cần thêm thông tin nào khác về bạn Gia Bảo, bạn cứ hỏi nhé!', sources=[ToolOutput(blocks=[TextBlock(block_type='text', text="[NodeWithScore(node=TextNode(id_='72598887-03af-4d58-ac00-f8f6df6b2c6c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e11aca32-8ee6-449d-966c-a2d32f5e2d78', node_type='4', metadata={}, hash='5abc971714457adc3ce6b453b1b88b8d685f7cb17b42c70eef37c8b55e063120')}, metadata_template='{key}: {v

In [4]:
chatEngine.load_old_docs()

ValueError: Unknown chat mode: CONDENSE_PLUS_CONTEXT

In [4]:
chatEngine.load_docs(new_doc)

In [67]:
chatEngine.index.vector_store.collection_name

In [5]:
chatEngine.docs

[]

In [ ]:
chatEngine.update_docs(new_doc)
chatEngine.update_docs(docs_a)

In [6]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core import StorageContext, VectorStoreIndex, Settings, Document
from llama_index.core.memory import ChatMemoryBuffer


class ChatEngine():
    def __init__(self, llm , docs: list[Document] = [],  collection_name: str = "docs", embed_model = HuggingFaceEmbedding(), token_limit :int = 4000, chroma_db_path: str = "./chroma_db"):
        
        self.docs = docs
        self.collection_name = collection_name
        self.chroma_db_path = chroma_db_path
        self.embed_model = embed_model
        self.token_limit = token_limit
        # Lưu tạm vào Chroma 
        chroma_client = chromadb.PersistentClient(path= chroma_db_path)
        
       

        # 3. Nếu collection đã tồn tại, load lại. Nếu chưa, tạo mới.
        chroma_collection = chroma_client.get_or_create_collection(collection_name)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        self.index = VectorStoreIndex.from_documents(
            docs, storage_context=storage_context, embed_model=embed_model, store_nodes_override=True
        )
        Settings.llm = llm                   # vô hiệu hóa LLM dùng OpenAI
        self.memory = ChatMemoryBuffer.from_defaults(token_limit=token_limit)
        self.chat_engine = self.index.as_chat_engine(chat_mode="context", memory=self.memory)
    def chat(self, message):
        if self.chat_engine:
            response = self.chat_engine.chat(message)
            return response
        else:
            return "Lỗi khi tạo mô hình"
    def chat2(self, message):
        if self.chat_engine:
            query_engine = self.index.as_query_engine(similarity_top_k=5)
            result = query_engine.query(message)
            print("🔎 Các đoạn được truy xuất:")
            for node in result.source_nodes:
                print("-" * 80)
                print(node.node.text[:500])
            return result.response
        else:
            return "Lỗi khi tạo mô hình"

    def update_docs(self, new_docs: list[Document]):
        self.docs.extend(new_docs)
        self.index = self.index.from_documents(self.docs)
        self.chat_engine = self.index.as_chat_engine(chat_mode='context', memory=self.memory)

        

from llama_index.core import VectorStoreIndex, Document
from llama_index.readers.file import PyMuPDFReader,DocxReader, PandasExcelReader 


def read_uploaded_file(file_path) -> list[Document] | None:

    filename = file_path.name if hasattr(file_path, "name") else str(file_path)
    ext = filename.lower().split('.')[-1]
    with open(file_path, "rb") as f:
        file_bytes = f.read()

    if ext == "pdf":
        pdf_loader = PyMuPDFReader()
        documents = pdf_loader.load(file_path=file_path)
    elif ext == "docx":
        docx_loader = DocxReader()
        documents = docx_loader.load_data(file  = file_path)
    elif ext in ["xls", "xlsx"]:
        excel_loader = PandasExcelReader()
        documents = excel_loader.load_data(file = file_path)
    elif ext == "txt":
        text = file_bytes.decode("utf-8")
        documents =  [Document(text=text, metadata={"filename": filename})]
    else:
        return None

    return documents


import os
from dotenv import load_dotenv
load_dotenv()
from llama_index.llms.openai_like import OpenAILike
from llama_index.llms.openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

# llm=OpenAILike(model="qwen/qwen3-coder:free",
#         api_key = api_key,
#         temperature = 0.0,

#         api_base = "https://openrouter.ai/api/v1",
#         is_chat_model=True,
#         is_function_calling_model=True, 
# )
llm = OpenAI(model="gpt-4.1-2025-04-14",
             temperature=0
             )
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", max_length=8000)
docs_a = read_uploaded_file(r"C:\Users\ACER\Desktop\nhan xet.docx")

new_doc = read_uploaded_file(r"C:\Users\ACER\Desktop\BTVN\BTVN GB12.xlsx")
chatEngine = ChatEngine(llm=llm,embed_model=embed_model, collection_name="final2", chroma_db_path="./db")
chatEngine.update_docs(new_doc)
chatEngine.update_docs(docs_a)





In [16]:
new_doc = read_uploaded_file(r"C:\Users\ACER\Desktop\BTVN\BTVN GB12.xlsx")
new_doc

[Document(id_='50e288e8-00a9-45bf-a217-dead54667885', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Unnamed: 0: BẢNG ĐIỂM BÀI TẬP VỀ NHÀ BUỔI 1 - 4 CỦA LỚP GB12, Unnamed: 1: , Unnamed: 2: , Unnamed: 3: , Unnamed: 4: , Unnamed: 5: , Unnamed: 6: , Unnamed: 7: , Unnamed: 8: , Unnamed: 9: , Unnamed: 10: , Unnamed: 11: \nUnnamed: 0: , Unnamed: 1: , Unnamed: 2: , Unnamed: 3: , Unnamed: 4: , Unnamed: 5: , Unnamed: 6: , Unnamed: 7: , Unnamed: 8: , Unnamed: 9: , Unnamed: 10: , Unnamed: 11: \nUnnamed: 0: STT, Unnamed: 1: Họ và tên học viên, Unnamed: 2: Buổi 1, Unnamed: 3: , Unnamed: 4: Buổi 2, Unnamed: 5: , Unnamed: 6: Buổi 3, Unnamed: 7: , Unnamed: 8: Buổi 4, Unnamed: 9: , Unnamed: 10: Buổi 6, Unnamed: 11: \nUnnamed: 0: , Unnamed: 1: , Unnamed: 2: Lý thuyết, Unnamed: 3: Thực hành, Unnamed: 4: Lý thuyết, Unname

In [17]:

file_path = r"C:\Users\ACER\Desktop\BTVN\BTVN GB12.xlsx"
excel_loader = PandasExcelReader()
documents = excel_loader.load_data(file = file_path)

In [18]:
documents

[Document(id_='eeef6e85-5100-4d97-ba7c-b7733de9bc7e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Unnamed: 0: BẢNG ĐIỂM BÀI TẬP VỀ NHÀ BUỔI 1 - 4 CỦA LỚP GB12, Unnamed: 1: , Unnamed: 2: , Unnamed: 3: , Unnamed: 4: , Unnamed: 5: , Unnamed: 6: , Unnamed: 7: , Unnamed: 8: , Unnamed: 9: , Unnamed: 10: , Unnamed: 11: \nUnnamed: 0: , Unnamed: 1: , Unnamed: 2: , Unnamed: 3: , Unnamed: 4: , Unnamed: 5: , Unnamed: 6: , Unnamed: 7: , Unnamed: 8: , Unnamed: 9: , Unnamed: 10: , Unnamed: 11: \nUnnamed: 0: STT, Unnamed: 1: Họ và tên học viên, Unnamed: 2: Buổi 1, Unnamed: 3: , Unnamed: 4: Buổi 2, Unnamed: 5: , Unnamed: 6: Buổi 3, Unnamed: 7: , Unnamed: 8: Buổi 4, Unnamed: 9: , Unnamed: 10: Buổi 6, Unnamed: 11: \nUnnamed: 0: , Unnamed: 1: , Unnamed: 2: Lý thuyết, Unnamed: 3: Thực hành, Unnamed: 4: Lý thuyết, Unname

In [29]:
chatEngine.update_docs(new_docs=new_doc)

C:\Users\ACER\AppData\Local\Temp\ipykernel_14964\4157999325.py:56: RuntimeWarning: coroutine 'VectorStoreIndex.ainsert_nodes' was never awaited
  self.index.ainsert_nodes(nodes)


In [2]:
chatEngine.index.docstore.docs

{'f6df4955-b570-474e-b279-8903f96ba41e': TextNode(id_='f6df4955-b570-474e-b279-8903f96ba41e', embedding=None, metadata={'file_name': 'nhan xet.docx'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2b485c39-6a5d-450d-96fe-c29f801e9a73', node_type='4', metadata={'file_name': 'nhan xet.docx'}, hash='dc42f77366115dcbfebe0a79ae68079bb172b7c69efdb9b56cd34233b115ca01'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c98d4436-aaec-4598-bdeb-1a68c1e187f3', node_type='1', metadata={}, hash='a9f7821c8a0085c9d94938079f8005ac0fdb1fca925a4c9f43fc5e5dc89bf52b')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Xin chào quý phụ huynh của lớp, sau đây em xin tổng hợp lại nội dung buổi học, slide cũng như đánh giá chung về quá trình học tập của lớp học GB230 trong tuần vừa qua: \n\n1/ Nội dung buổi học: \n\n-\tTìm hiểu và làm quen với giao diện phần mềm GameMaker;\n\n-\tCác khái niệm cơ c

In [3]:
response = chatEngine.chat("Bạn Hà Gia Bảo lớp GI12 thực hiện bài kiểm tra với điểm trắc nghiệm, thực hành là bao nhiêu còn thiếu những bài tập của buổi học nào?")
print(response)
print('*'*100)
response = chatEngine.chat("lớp GB228 học ra sao?")
print(response)

Dựa trên thông tin bạn cung cấp, hiện tại trong file **nhan xet.docx** không có dữ liệu chi tiết về điểm trắc nghiệm, điểm thực hành hay danh sách bài tập còn thiếu của từng học sinh, bao gồm bạn Hà Gia Bảo lớp GI12. File chỉ tổng hợp nội dung buổi học, link video, bài tập về nhà và nhận xét chung của lớp GB230.

**Để trả lời chính xác các câu hỏi của bạn về Hà Gia Bảo lớp GI12, bạn cần cung cấp thêm các thông tin sau:**
- Bảng điểm hoặc kết quả kiểm tra của Hà Gia Bảo (nếu có).
- Danh sách bài tập đã giao và bài tập Hà Gia Bảo đã nộp.

**Tuy nhiên, dựa trên nội dung buổi học gần nhất (GB230), các bài tập được giao là:**
- Thực hiện bài tập trên nền tảng Denise qua đường link: https://base.mindx.edu.vn/login?callbackUrl=https://denise.mindx.edu.vn

**Nếu Hà Gia Bảo chưa hoàn thành bài tập trên nền tảng Denise, thì đây là bài tập còn thiếu của buổi học vừa qua.**

**Tóm tắt:**
- **Điểm trắc nghiệm, thực hành:** Chưa có thông tin cụ thể.
- **Bài tập còn thiếu:** Nếu chưa làm bài tập trên